In [1]:
%pip install pymongo
%pip install chromadb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 2.9 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 379.6 kB/s eta 0:00:000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 672.1 kB/s eta 0:00:00 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 130.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 397.3 kB/s eta 0:00:0000:01
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached idna-3.10-p

In [1]:
with open(".env", "r") as env:
    env_variables = env.readlines()

for line in env_variables:
    if "MONGO_CLIENT" in line:
        mongoclient = line.strip().split("=")[1]

In [ ]:
from pymongo import MongoClient
import chromadb

client = MongoClient(mongoclient)

In [47]:
db = client.get_database('rnr-sys-chatbot')
collection = db.get_collection('home') 

documents = collection.find()
text_data = [doc for doc in documents]

In [4]:
%pip install huggingface-hub
%pip -q install langchain
%pip -q install langchain_community
%pip -q install tiktoken
%pip install langchainhub


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [83]:
import os
import numpy as np
from getpass import getpass
from langchain import hub
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document

In [ ]:
with open(".env", "r") as env:
    env_variables = env.readlines()

for line in env_variables:
    if "HUGGINGFACE_ACCESS_TOKEN" in line:
        hfapi_key = line.strip().split("=")[1]
    if "HUGGINGFACE_ACCESS_TOKEN_TEXT_GEN" in line:
        hfapi_key_text_gen = line.strip().split("=")[1]
    if "OPENAI_API_KEY" in line:
        openai_key = line.strip().split("=")[1]
    if "OPENAI_MODEL_ID" in line:
        openai_model = line.strip().split("=")[1]

os.environ["HF_TOKEN"] = hfapi_key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfapi_key
os.environ["OPENAI_API_KEY"] = openai_key

vector_db = "vectordatabase"

In [4]:
text_data[0]

{'_id': '94877768-b065-4ed4-bee2-7c764e447e7c',
 'content': {'Title': 'RNR Systems -',
  'Subtitle': 'Reimagining and Revolutionary Systems Discover cutting-edge visual solutions that elevate your technology interactions, captivating audiences with digital projections, LED displays, and immersive holographic presentations tailored for impactful brand experiences. LED Products projection systems Innovative SolutionsState-of-the-art technology designed to engage and inspire your audience. Custom ExperiencesTailored visual solutions that fit your unique brand and event needs.',
  'Content': 'Skip to content\n\n  * Home\n  * About\n  * LED Products\n  * Projection Systems\n  * Creative Hoardings\n  * Contact\n\nMain Menu\n\n  * Home\n  * About\n  * LED Products\n  * Projection Systems\n  * Creative Hoardings\n  * Contact\n\n## Reimagining and Revolutionary Systems\n\n##### Discover cutting-edge visual solutions that elevate your technology\ninteractions, captivating audiences with digital 

In [48]:
documents

In [49]:
documents = collection.find()
processed_documents = []
for doc in documents:
    text_content = doc.get("content", {}).get("Content", "")
    metadata = dict()
    metadata['Title'] = doc.get("content", {}).get("Title", "")
    metadata['Subtitle'] = doc.get("content", {}).get("Subtitle", "")
    metadata['language'] = doc.get("content", {}).get("language", "")
    metadata['link'] = doc.get("link", {})
    metadata['platform'] = doc.get('platform', {})
    if len(text_content):
        document = Document(
            page_content=text_content,
            metadata=metadata
        )
        processed_documents.append(document)

In [53]:
processed_documents[0]

Document(metadata={'Title': 'RNR Systems -', 'Subtitle': 'Reimagining and Revolutionary Systems Discover cutting-edge visual solutions that elevate your technology interactions, captivating audiences with digital projections, LED displays, and immersive holographic presentations tailored for impactful brand experiences. LED Products projection systems Innovative SolutionsState-of-the-art technology designed to engage and inspire your audience. Custom ExperiencesTailored visual solutions that fit your unique brand and event needs.', 'language': 'en-US', 'link': 'https://rnrsystems.in/', 'platform': '/'}, page_content='Skip to content\n\n  * Home\n  * About\n  * LED Products\n  * Projection Systems\n  * Creative Hoardings\n  * Contact\n\nMain Menu\n\n  * Home\n  * About\n  * LED Products\n  * Projection Systems\n  * Creative Hoardings\n  * Contact\n\n## Reimagining and Revolutionary Systems\n\n##### Discover cutting-edge visual solutions that elevate your technology\ninteractions, captiv

In [52]:
from langchain_community.vectorstores.utils import filter_complex_metadata

filtered_documents = [filter_complex_metadata(doc) for doc in processed_documents]

AttributeError: 'tuple' object has no attribute 'metadata'

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,
                    chunk_overlap=20,
                    length_function=len,
                    is_separator_regex=False)

In [10]:
chunks = text_splitter.split_documents(processed_documents)

In [12]:
len(chunks)

899

In [11]:
chunks

[Document(metadata={'Title': 'RNR Systems -', 'Subtitle': 'Reimagining and Revolutionary Systems Discover cutting-edge visual solutions that elevate your technology interactions, captivating audiences with digital projections, LED displays, and immersive holographic presentations tailored for impactful brand experiences. LED Products projection systems Innovative SolutionsState-of-the-art technology designed to engage and inspire your audience. Custom ExperiencesTailored visual solutions that fit your unique brand and event needs.', 'language': 'en-US', 'link': 'https://rnrsystems.in/', 'platform': '/'}, page_content='Skip to content'),
 Document(metadata={'Title': 'RNR Systems -', 'Subtitle': 'Reimagining and Revolutionary Systems Discover cutting-edge visual solutions that elevate your technology interactions, captivating audiences with digital projections, LED displays, and immersive holographic presentations tailored for impactful brand experiences. LED Products projection systems 

In [56]:
for chunk in chunks:
    if chunk.metadata == "":
        print(chunk)

In [59]:
from langchain_community.vectorstores.utils import filter_complex_metadata

filtered_chunks = filter_complex_metadata(chunks)

In [66]:
len(filtered_chunks)

899

In [14]:
%pip install langchain_openai
%pip install langchain_chroma
%pip install scikit-learn
%pip install plotly
%pip install numpy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 398.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 7.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 527.6 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 1.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB

In [28]:
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [29]:
embeddings = OpenAIEmbeddings()

In [75]:
if os.path.exists(vector_db):
    Chroma(persist_directory=vector_db, embedding_function=embeddings).delete_collection()

In [76]:
vector_store = Chroma.from_documents(documents=filtered_chunks, embedding=embeddings, persist_directory=vector_db)
print(f"Vectorstore created with {vector_store._collection_count} documents")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-proj**********************************************************************************************************************************************************-EA". You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [34]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
if os.path.exists(vector_db):
    Chroma(persist_directory=vector_db, embedding_function=hf).delete_collection()

In [77]:
vector_store = Chroma.from_documents(documents=filtered_chunks, embedding=hf, persist_directory=vector_db)
print(f"Vectorstore created with {len(filtered_chunks)} documents")

Vectorstore created with 899 documents


In [80]:
collections = vector_store._collection
sample_embeddings = collections.get(limit=1, include=['embeddings'])
print(sample_embeddings)
print(f"""The dimension of embeddings is {len(sample_embeddings["embeddings"][0])}""")

{'ids': ['9d0003bf-3745-4fa0-ad87-947b266fffdd'], 'embeddings': array([[ 1.94002949e-02,  2.51357667e-02,  3.61100101e-04,
         2.38927435e-02,  8.09205994e-02,  7.51753598e-02,
        -6.77996967e-03, -7.57555738e-02,  6.94532171e-02,
         4.05106039e-05,  3.55535075e-02,  2.91753281e-02,
         4.35812250e-02, -1.99165083e-02, -6.38249423e-03,
        -3.32584940e-02,  6.71323985e-02,  5.31635545e-02,
        -2.94816475e-02, -7.34007061e-02, -9.36502665e-02,
         4.09015343e-02,  2.79310104e-02,  7.62679502e-02,
        -3.27568986e-02,  6.88617723e-03, -8.86818618e-02,
         1.25126280e-02,  2.19445024e-02, -5.29346578e-02,
         2.21199337e-02, -3.68373059e-02,  1.37964776e-02,
         6.15768973e-03,  1.81039013e-02,  2.39696261e-03,
        -3.28859203e-02, -6.29043905e-03,  3.87814902e-02,
         1.25298556e-02,  6.54679071e-03, -6.44783536e-03,
        -5.42870089e-02,  2.38358770e-02,  1.48886582e-04,
        -2.29658782e-02, -6.82400987e-02, -6.434542

In [ ]:
os.environ["HF_TOKEN"] = hfapi_key_text_gen
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hfapi_key_text_gen

In [96]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens = 512,
    top_k = 30,
    temperature = 0.1,
    repetition_penalty = 1.03,
    huggingfacehub_api_token=hfapi_key,
)
"""chat_llm = HuggingFaceEndpoint(
    endpoint_url="http://localhost:8010/",
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    huggingfacehub_api_token="my-api-key"
)"""



"""llm = HuggingFaceEndpoint(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)"""

chat = ChatHuggingFace(llm=llm, verbose=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [97]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vector_store.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [98]:
query = "What is LED solution ?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

/home/himanshu/Projects/RAG/RNR-System-Chatbot/.chatbot/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.



 LED (Light Emitting Diode) solution refers to the use of LED (Light Emitting Diode) technology to provide lighting or display options in various applications. LED solutions can be used for indoor or outdoor purposes and offer high-quality, energy-efficient, and long-lasting lighting options. They are commonly used in advertising displays, signage, architectural lighting, and other applications where bright, long-lasting, and energy-efficient lighting is required.


In [90]:
llm = ChatOpenAI(temperature=0.1, model=openai_model)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
retriever = vector_store.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/tmp/ipykernel_119543/1887807588.py:2: LangChainDeprecationWarning:

Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/



In [91]:
query = "What is LED solution ?"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-proj**********************************************************************************************************************************************************-EA". You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [99]:
%pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 676.0 kB/s eta 0:00:00 0:00:01
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 1.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 11.8 MB/s eta 0:00:0000:0100:01
Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.9/507.9 kB 3.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 2.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install

In [100]:
def chat(message, history):
    result = conversation_chain.invoke({"question":message})
    return result["answer"]

In [101]:
import gradio as gr

view = gr.ChatInterface(chat).launch()

/home/himanshu/Projects/RAG/RNR-System-Chatbot/.chatbot/lib/python3.11/site-packages/gradio/chat_interface.py:334: UserWarning:

The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.



* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/himanshu/Projects/RAG/RNR-System-Chatbot/.chatbot/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.

/home/himanshu/Projects/RAG/RNR-System-Chatbot/.chatbot/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning:

'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingf